In [11]:
from gensim import corpora, models, similarities
import gensim

In [12]:
import json 
import gzip 
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g: 
        yield eval(l)

import gensim
class MyReviews():


    def __iter__(self):

        review_item_iterator = iter(parse("../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz"))
        #for review_item in parse("../../download/reviews_Cell_Phones_and_Accessories.json.gz"): 
        for it in range(100000):
            if it%10000==0:
                print("it="+str(it))
            review_item = next(review_item_iterator)
            #print(review_item)
        #for review_item in self.reviews_cursor:


            reviewText = review_item["reviewText"]
            label = review_item["asin"] + "_" + review_item["reviewerID"] 
            preprocess_reviewText = gensim.utils.simple_preprocess(reviewText)
                  

            #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
            yield preprocess_reviewText

In [18]:
import gensim
def myReviews(catFilter = None, asin = None, concatenate = False):
    preprocess_reviewText = []
    review_item_iterator = iter(parse("../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz"))
    it = 0
    conc_count = 0
    for review_item in review_item_iterator:
        it += 1
    #for it in range(1000000):
        if it%50000==0:
            print("it="+str(it))
        review_item = next(review_item_iterator)
        #print(review_item)
    #for review_item in self.reviews_cursor:


        reviewText = review_item["reviewText"]
        label = review_item["asin"] + "_" + review_item["reviewerID"] 
        preprocess_reviewText_i = gensim.utils.simple_preprocess(reviewText)


        #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
        if (catFilter == None or (catFilter in review_item["categories"])) and (asin == None or (asin == review_item["asin"])):
            if concatenate==False:
                yield preprocess_reviewText_i
            else:
                conc_count += 1
                preprocess_reviewText.extend(preprocess_reviewText_i)
                print("Concatenated count: "+str(conc_count))
    if concatenate == True:
        yield preprocess_reviewText

In [14]:
import os
dictionaryFileName = '../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.dict'
if os.path.isfile(dictionaryFileName)==False:
    dictionary = corpora.Dictionary(myReviews("Cell Phones"))
    from six import iteritems
    once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
    dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once
    dictionary.compactify()  # remove gaps in id sequence after words that were removed
    dictionary.save(dictionaryFileName)  # store the dictionary, for future reference
else:
    dictionary = corpora.Dictionary.load(dictionaryFileName)

print(dictionary)

Dictionary(28586 unique tokens: ['product', 'came', 'exactly', 'as', 'described']...)


In [15]:
def myCorpus(catFilter = None, asin = None, concatenate = False):
    for review in myReviews(catFilter,asin,concatenate):
    # assume there's one document per line, tokens separated by whitespace
        yield dictionary.doc2bow(review)

In [16]:
import os
tfidfFileName ='../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat_corpus.tfidf'
if os.path.isfile(tfidfFileName)==False:
    tfidf = models.TfidfModel(myCorpus("Cell Phones"))
    tfidf.save(tfidfFileName)
else:
    tfidf = models.TfidfModel.load(tfidfFileName)

print(tfidf)

TfidfModel(num_docs=69127, num_nnz=4450875)


In [ ]:
apple_5s_corpus = myCorpus(asin="B00F3J4B5S",concatenate=True)
apple_5s_corpus_tfidf = tfidf[apple_5s_corpus]

In [ ]:
import pickle

with open('../../download/word2vec/apple_5s_review.corpus', 'wb') as fp:
    pickle.dump(apple_5s_corpus, fp)

In [ ]:
print(apple_5s_corpus_tfidf)

In [ ]:
print(next(apple_5s_corpus))